<center>
<h1 style="font-size: 50px; font-weight: bold; color:sandybrown">OC SWITRS R Data Processing</h1>

<div style="font-size: 40px; font-weight: bold; color: sandybrown">ArcGIS Hot Spot Analysis</div>
<div style="font-size: 30px; font-weight: bold; color: sandybrown">v.1, January 2025</div>
</center>

----

## <font color="orangered">**1. Referencing Libraries and Initialization**</font>

### <font color="blue">**1.1. Preliminaries**</font>

Instantiating python libraries for the project

In [70]:
# Import Python libraries
import os, json, pytz, math, arcpy, arcgis
from datetime import date, time, datetime, timedelta, tzinfo, timezone
from tqdm.notebook import trange, tqdm, tqdm_notebook
import pandas as pd
import numpy as np
from arcpy import metadata as md

# important as it "enhances" Pandas by importing these classes (from ArcGIS API for Python)
from arcgis.features import GeoAccessor, GeoSeriesAccessor


### <font color="blue">**1.2. Project and Workspace Variables**</font>

Define and maintain project, workspace, ArcGIS, and data-related variables.

#### <font color="darkgreen">Project and Geodatabase Paths</font>

Project and ArcGIS Pro project paths

In [71]:
# Environment variables for OneDrive path
onedrive_path = os.getenv("OneDriveCommercial")

# OC Switrs project path
project_path = os.path.join(onedrive_path, "Documents", "OCSWITRS")

# OC Switrs ArcGIS Pro project path
agp_path = os.path.join(project_path, "AGPSWITRS")

print(f"Project Directories:\n- Project Path:{project_path}\n- ArcGIS Pro Project Path: {agp_path}")


Project Directories:
- Project Path:C:\Users\ktalexan\OneDrive - County of Orange\Documents\OCSWITRS
- ArcGIS Pro Project Path: C:\Users\ktalexan\OneDrive - County of Orange\Documents\OCSWITRS\AGPSWITRS


#### <font color="darkgreen">ArcGIS Pro Paths</font>

ArcGIS Pro related paths and variables

In [151]:
# ArcGIS Pro project name and path
project_aprx = "AGPSWITRS.aprx"
project_aprx_path = os.path.join(agp_path, project_aprx)

# ArcGIS Pro project geodatabase and path
gdb = "AGPSWITRS.gdb"
gdb_path = os.path.join(agp_path, gdb)

# Current ArcGIS pro project
aprx = arcpy.mp.ArcGISProject('CURRENT')

# Close all map views
aprx.closeViews()

# Current ArcGIS pro project maps
map_main = aprx.listMaps("MainMap")[0]
map_hotspots = aprx.listMaps("HotSpotsMap")[0]

# Current ArcGIS pro project layouts
layout_main = aprx.listLayouts("MainLayout")[0]
layout_hotsponts = aprx.listLayouts("HotSpotsLayout")[0]

# Current ArcGIS workspace (arcpy)
arcpy.env.workspace = gdb_path
workspace = arcpy.env.workspace

# Enable overwriting existing outputs
arcpy.env.overwriteOutput = True

#### <font color="darkgreen">Folder Paths</font>

Project folder paths

In [73]:
# Raw data folder path
raw_data_path = os.path.join(project_path, "Data", "Raw")

# Layers folder path
layers_path = os.path.join(project_path, "Layers")

# Notebooks folder path
notebooks_path = os.path.join(project_path, "Notebooks")

# Geodatabase feature directory paths
gdb_raw_data_path = os.path.join(workspace, "RawData")  # raw data
gdb_supporting_data_path = os.path.join(
    workspace, "SupportingData")  # supporting data
gdb_analysis_data_path = os.path.join(
    workspace, "AnalysisData")  # analysis data
gdb_hotspot_data_path = os.path.join(workspace, "HotSpotData")  # hotspot data

# Codebook Json folder path
codebook_path = os.path.join(project_path, "Data", "codebook", "cb.json")


#### <font color="darkgreen">Data Folder Paths</font>

Data folder paths and contents

The most current data files cover the periods from 01/01/2013 to 09/30/2024. The data files are already processed in the R scripts and imported into the project's geodatabase.

In [74]:
# add the date 01/01/2013 to a new python datetime object named 'rawDateStart'
raw_date_start = datetime(2013, 1, 1)

# add the date 06/30/2024 to a new python datetime object named 'rawDateEnd'
raw_date_end = datetime(2024, 9, 30)

# Define time and date variables
time_zone = pytz.timezone("US/Pacific")
today = datetime.now(time_zone)
last_update_date = today.strftime("%B %d, %Y")
last_update_datetime = today.strftime("%A %B %d, %Y, %I:%M %p (%Z)")


#### <font color="darkgreen">Geodatabase Feature Classes Paths</font>

ArcGIS Pro geodatabase data paths and feature classes

In [75]:
# Paths to raw data feature classes
crashes_path = os.path.join(gdb_raw_data_path, "crashes")
parties_path = os.path.join(gdb_raw_data_path, "parties")
victims_path = os.path.join(gdb_raw_data_path, "victims")
collisions_path = os.path.join(gdb_raw_data_path, "collisions")

# Paths to supporting data feature classes
boundaries_path = os.path.join(gdb_supporting_data_path, "boundaries")
cities_path = os.path.join(gdb_supporting_data_path, "cities")
roads_path = os.path.join(gdb_supporting_data_path, "roads")
usc2020blk_path = os.path.join(gdb_supporting_data_path, "usc2020blk")


ArcGIS Pro geodatabase supporting data paths and feature classes

In [76]:
# Display all information
print("Key Project Information")
print(f"\n\t- Name {project_aprx}\n\t- Path: {project_aprx_path}\n\t- Project Path: {project_path}\n\t- Workspace: {workspace}\n\t- Geodatabase: {gdb}\n\t- Geodatabase Path: {gdb_path}")
print("\nProject Directories:")
print(
    f"\n\t- Raw Data: {raw_data_path}\n\t- Layers: {layers_path}\n\t- Notebooks: {notebooks_path}")
print("\nRaw Feature Classes:")
print(f"\n\t- Crashes: {crashes_path}\n\t- Parties: {parties_path}\n\t- Victims: {victims_path}\n\t- Collisions: {collisions_path}")
print("\nSupporting Feature Classes:")
print(
    f"\n\t- Boundaries: {boundaries_path}\n\t- Cities: {cities_path}\n\t- Roads: {roads_path}\n\t- USC 2020 Blocks: {usc2020blk_path}")
print("\nOther Supporting Data")
print(f"\n\t- Codebook: {codebook_path}")


Key Project Information

	- Name AGPSWITRS.aprx
	- Path: C:\Users\ktalexan\OneDrive - County of Orange\Documents\OCSWITRS\AGPSWITRS\AGPSWITRS.aprx
	- Project Path: C:\Users\ktalexan\OneDrive - County of Orange\Documents\OCSWITRS
	- Workspace: C:\Users\ktalexan\OneDrive - County of Orange\Documents\OCSWITRS\AGPSWITRS\AGPSWITRS.gdb
	- Geodatabase: AGPSWITRS.gdb
	- Geodatabase Path: C:\Users\ktalexan\OneDrive - County of Orange\Documents\OCSWITRS\AGPSWITRS\AGPSWITRS.gdb

Project Directories:

	- Raw Data: C:\Users\ktalexan\OneDrive - County of Orange\Documents\OCSWITRS\Data\Raw
	- Layers: C:\Users\ktalexan\OneDrive - County of Orange\Documents\OCSWITRS\Layers
	- Notebooks: C:\Users\ktalexan\OneDrive - County of Orange\Documents\OCSWITRS\Notebooks

Raw Feature Classes:

	- Crashes: C:\Users\ktalexan\OneDrive - County of Orange\Documents\OCSWITRS\AGPSWITRS\AGPSWITRS.gdb\RawData\crashes
	- Parties: C:\Users\ktalexan\OneDrive - County of Orange\Documents\OCSWITRS\AGPSWITRS\AGPSWITRS.gdb\RawDa

#### <font color="darkgreen">Current Map Operations</font>

Remove all layers from the Hot Spots Map

In [79]:
for lyr in map_hotspots.listLayers():
    if "World Topographic Map" not in lyr.name:
        map_hotspots.removeLayer(lyr)

Close all active views and open the active map

In [80]:
# Close all map views
aprx.closeViews()

# Open the main map window
map_hotspots.openView()

# set the main map as active map
map = aprx.activeMap


Set the working directory to the root of the project geodatabase.

In [81]:
# Current ArcGIS workspace (arcpy)
arcpy.env.workspace = gdb_path
workspace = arcpy.env.workspace

# Enable overwriting existing outputs
arcpy.env.overwriteOutput = True


## <font color="orangered">**2. Map and Geodatabase Operations**</font>

List all the layers on the main map

In [82]:
# List all layers in the map
for lyr in map_main.listLayers():
    print(lyr.name)
    

OCSWITRS Collisions
OCSWITRS Crashes
OCSWITRS Parties
OCSWITRS Victims
OCSWITRS Roads
OCSWITRS Census 2020 Blocks
OCSWITRS Cities
OCSWITRS Boundaries
World Topographic Map


Turn off visibility of the layers in the map

In [83]:
# Iterate through the data layers list
for lyr in map_main.listLayers():
    if "OCSWITRS" in lyr.name:
        # Set the visibility of the layer to True
        lyr.visible = False


Save the ArcGIS Pro project

In [84]:
# Save the ArcGIS Pro analysis project
aprx.save()


### <font color="blue">**2.1. Adding Symbology from Layers**</font>

In [85]:
# List all the layers in the map
main_map_lyrs = map_main.listLayers()

# Get the layers from the map
collisions_lyr = next((mlyr for mlyr in main_map_lyrs if mlyr.name == "OCSWITRS Collisions"), False)
crashes_lyr = next((mlyr for mlyr in main_map_lyrs if mlyr.name == "OCSWITRS Crashes"), False)
parties_lyr = next((mlyr for mlyr in main_map_lyrs if mlyr.name == "OCSWITRS Parties"), False)
victims_lyr = next((mlyr for mlyr in main_map_lyrs if mlyr.name == "OCSWITRS Victims"), False)
roads_lyr = next((mlyr for mlyr in main_map_lyrs if mlyr.name == "OCSWITRS Roads"), False)
usc2020blk_lyr = next((mlyr for mlyr in main_map_lyrs if mlyr.name == "OCSWITRS Census 2020 Blocks"), False)
cities_lyr = next((mlyr for mlyr in main_map_lyrs if mlyr.name == "OCSWITRS Cities"), False)
boundaries_lyr = next((mlyr for mlyr in main_map_lyrs if mlyr.name == "OCSWITRS Boundaries"), False)


### <font color="blue">**2.2. Create Major Roads**</font>

Separate the primary and secondary roads from the local roads

In [89]:
# Output feature for the Major roads
roads_major_path = os.path.join(gdb_analysis_data_path, "roads_major")

# Select the major (primary and secondary) roads from the roads feature class
arcpy.analysis.Select(
    in_features = roads_path,
    out_feature_class = roads_major_path,
    where_clause = "road_cat = 'Primary' Or road_cat = 'Secondary'"
)

<Result 'C:\\Users\\ktalexan\\OneDrive - County of Orange\\Documents\\OCSWITRS\\AGPSWITRS\\AGPSWITRS.gdb\\AnalysisData\\roads_major'>

Map layer operations

In [90]:
# Remove the output layer
map.removeLayer(map.listLayers("roads_major")[0])

# Change the feature class alias for the Major Roads
arcpy.AlterAliasName(roads_major_path, "Major Roads")

# Add the major roads layer to the Hot Spots map
roads_major_lyr = map_hotspots.addDataFromPath(roads_major_path)
roads_major_lyr.visible = False

# Apply the symbology for the major roads layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = roads_major_lyr,
    in_symbology_layer = os.path.join(layers_path, "Major Roads.lyrx"),
    symbology_fields = "VALUE_FIELD road_cat road_cat",
    update_symbology = "MAINTAIN"
)

<Result 'Major Roads'>

### <font color="blue">**2.3. Create Road Buffers**</font>

Create road buffers for the primary and secondary roads

In [92]:
# Output feature for the major road buffers
roads_major_buffers_path = os.path.join(gdb_analysis_data_path, "roads_major_buffers")

# Buffer the major roads by 250 meters
arcpy.analysis.Buffer(
    in_features = roads_major_path,
    out_feature_class = roads_major_buffers_path,
    buffer_distance_or_field = "250 Meters",
    line_side = "FULL",
    line_end_type = "FLAT",
    dissolve_option = "ALL",
    dissolve_field = None,
    method = "PLANAR"
)

<Result 'C:\\Users\\ktalexan\\OneDrive - County of Orange\\Documents\\OCSWITRS\\AGPSWITRS\\AGPSWITRS.gdb\\AnalysisData\\roads_major_buffers'>

Map layer operations

In [93]:
# Remove the newly created layer from map
map.removeLayer(map.listLayers("roads_major_buffers")[0])

# Change the feature class alias for the Major Road Buffers
arcpy.AlterAliasName(roads_major_buffers_path, "Major Road Buffers")

# Add the major road buffers layer to the Hot spots map
roads_major_buffers_lyr = map_hotspots.addDataFromPath(roads_major_buffers_path)
roads_major_buffers_lyr.visible = False


### <font color="blue">**2.4. Summarize Census Blocks**</font>

Create a summary for each of the Census blocks that contains statistics and counts of crash collision data

In [94]:
# Create a path for the new summarized US 2020 Census Blocks feature class
usc2020blk_sum_path = os.path.join(gdb_analysis_data_path, "usc2020blk_sum")

arcpy.analysis.SummarizeWithin(
    in_polygons = usc2020blk_path,
    in_sum_features = crashes_path,
    out_feature_class = usc2020blk_sum_path,
    keep_all_polygons = "KEEP_ALL",
    sum_fields = [
        ["crash_tag", "Sum"], ["party_count", "Sum"], ["victim_count", "Sum"], ["number_killed", "Sum"], 
        ["number_inj", "Sum"], ["count_severe_inj", "Sum"], ["count_visible_inj", "Sum"],
        ["count_complaint_pain", "Sum"], ["count_car_killed", "Sum"], ["count_car_inj", "Sum"],
        ["count_ped_killed", "Sum"], ["count_ped_inj", "Sum"], ["count_bic_killed", "Sum"],
        ["count_bic_inj", "Sum"], ["count_mc_killed", "Sum"], ["count_mc_inj", "Sum"],
        ["coll_severity_num", "Mean"], ["coll_severity_rank_num", "Mean"]
    ]
)

<Result 'C:\\Users\\ktalexan\\OneDrive - County of Orange\\Documents\\OCSWITRS\\AGPSWITRS\\AGPSWITRS.gdb\\AnalysisData\\usc2020blk_sum'>

Map layer operations

In [95]:
# Remove the newly created layer from map
map.removeLayer(map.listLayers("usc2020blk_sum")[0])

# Change the feature class alias for the Summarized US Census 2020 Blocks
arcpy.AlterAliasName(usc2020blk_sum_path, "OCSWITRS Census 2020 Blocks Summary")

# Add the Census 2020 Blocks summary layer to the hot spots map
usc2020blk_sum_lyr = map_hotspots.addDataFromPath(usc2020blk_sum_path)
usc2020blk_sum_lyr.visible = False

# Apply the symbology for the Census 2020 Blocks summary layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = usc2020blk_sum_lyr,
    in_symbology_layer = os.path.join(layers_path, "OCSWITRS Census 2020 Blocks.lyrx"),
    symbology_fields = "VALUE_FIELD population_total population_total;EXCLUSION_CLAUSE_FIELD population_total population_total",
    update_symbology = "MAINTAIN"
)

<Result 'OCSWITRS Census 2020 Blocks Summary'>

### <font color="blue">**2.5. Summarize Cities**</font>

Create a summary for each of the cities that contains statistics and counts of crash collision data

In [96]:
# Create a path for the new summarized US 2020 Census Blocks feature class
cities_sum_path = os.path.join(gdb_analysis_data_path, "cities_sum")

# Summarize the data within the cities from the crashes data
arcpy.analysis.SummarizeWithin(
    in_polygons = cities_path,
    in_sum_features = crashes_path,
    out_feature_class = cities_sum_path,
    keep_all_polygons = "KEEP_ALL",
    sum_fields=[
        ["crash_tag", "Sum"], ["party_count", "Sum"], ["victim_count", "Sum"], ["number_killed", "Sum"], 
        ["number_inj", "Sum"], ["count_severe_inj", "Sum"], ["count_visible_inj", "Sum"],
        ["count_complaint_pain", "Sum"], ["count_car_killed", "Sum"], ["count_car_inj", "Sum"],
        ["count_ped_killed", "Sum"], ["count_ped_inj", "Sum"], ["count_bic_killed", "Sum"],
        ["count_bic_inj", "Sum"], ["count_mc_killed", "Sum"], ["count_mc_inj", "Sum"],
        ["coll_severity_num", "Mean"], ["coll_severity_rank_num", "Mean"]
    ]
)

<Result 'C:\\Users\\ktalexan\\OneDrive - County of Orange\\Documents\\OCSWITRS\\AGPSWITRS\\AGPSWITRS.gdb\\AnalysisData\\cities_sum'>

Map layer operations

In [97]:
# Remove the newly created layer from map
map.removeLayer(map.listLayers("cities_sum")[0])

# Change the feature class alias for the Summarized Cities
arcpy.AlterAliasName(cities_sum_path, "OCSWITRS Cities Summary")

# Add the Census 2020 Blocks summary layer to the hot spots map
cities_sum_lyr = map_hotspots.addDataFromPath(cities_sum_path)
cities_sum_lyr.visible = False

# Apply the symbology for the Cities summary layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = cities_sum_lyr,
    in_symbology_layer = os.path.join(layers_path, "OCSWITRS Cities.lyrx"),
    symbology_fields = "VALUE_FIELD city_pop_dens city_pop_dens;COLOR_EXPRESSION_FIELD city_pop_dens city_pop_dens",
    update_symbology = "MAINTAIN"
)

<Result 'OCSWITRS Cities Summary'>

### <font color="blue">**2.6. Summarize Major Road Buffers**</font>

Create a summary for each of the road buffers that contains statistics and counts of crash collision data

In [105]:
# Create a path for the new summarized major road buffers feature class
roads_major_buffers_sum_path = os.path.join(gdb_analysis_data_path, "roads_major_buffers_sum")

arcpy.analysis.SummarizeWithin(
    in_polygons = roads_major_buffers_path,
    in_sum_features = crashes_path,
    out_feature_class = roads_major_buffers_sum_path,
    keep_all_polygons = "KEEP_ALL",
    sum_fields=[
        ["crash_tag", "Sum"], ["party_count", "Sum"], ["victim_count", "Sum"], ["number_killed", "Sum"], 
        ["number_inj", "Sum"], ["count_severe_inj", "Sum"], ["count_visible_inj", "Sum"],
        ["count_complaint_pain", "Sum"], ["count_car_killed", "Sum"], ["count_car_inj", "Sum"],
        ["count_ped_killed", "Sum"], ["count_ped_inj", "Sum"], ["count_bic_killed", "Sum"],
        ["count_bic_inj", "Sum"], ["count_mc_killed", "Sum"], ["count_mc_inj", "Sum"],
        ["coll_severity_num", "Mean"], ["coll_severity_rank_num", "Mean"]
    ]
)

<Result 'C:\\Users\\ktalexan\\OneDrive - County of Orange\\Documents\\OCSWITRS\\AGPSWITRS\\AGPSWITRS.gdb\\AnalysisData\\roads_major_buffers_sum'>

Map layer operations

In [106]:
# Remove the newly created layer from map
map.removeLayer(map.listLayers("roads_major_buffers_sum")[0])

# Change the feature class alias for the Summarized Major Road Buffers
arcpy.AlterAliasName(roads_major_buffers_sum_path, "Major Road Buffers Summary")

# Add the Major road buffers summary layer to the hot spots map
roads_major_buffers_sum_lyr = map_hotspots.addDataFromPath(roads_major_buffers_sum_path)
roads_major_buffers_sum_lyr.visible = False

# move the layer
map.moveLayer(roads_major_buffers_lyr, roads_major_buffers_sum_lyr, insert_position = "BEFORE")

True

### <font color="blue">**2.7. Process Road Lines and Segments**</font>

#### <font color="darkgreen">Points 1,000 ft along major road lines</font>

Generate points every 1,000 feet along the major road lines

In [112]:
# Create a path for the new summarized major road buffers feature class
roads_major_points_along_lines_path = os.path.join(gdb_analysis_data_path, "roads_major_points_along_lines")

arcpy.management.GeneratePointsAlongLines(
    Input_Features = roads_major_path,
    Output_Feature_Class = roads_major_points_along_lines_path,
    Point_Placement = "DISTANCE",
    Distance = "1000 Feet",
    Percentage = None,
    Include_End_Points = "NO_END_POINTS",
    Add_Chainage_Fields = "NO_CHAINAGE",
    Distance_Field=None,
    Distance_Method = "PLANAR"
)

<Result 'C:\\Users\\ktalexan\\OneDrive - County of Orange\\Documents\\OCSWITRS\\AGPSWITRS\\AGPSWITRS.gdb\\AnalysisData\\roads_major_points_along_lines'>

Map Layer Operations

In [113]:
# Remove the newly created layer from map
map.removeLayer(map.listLayers("roads_major_points_along_lines")[0])

# change the feature class alias for the Major Road Points Along Lines
arcpy.AlterAliasName(roads_major_points_along_lines_path, "Major Road Points Along Lines")

# Add the major road points along lines layer to the hot spots map
roads_major_points_along_lines_lyr = map_hotspots.addDataFromPath(roads_major_points_along_lines_path)
roads_major_points_along_lines_lyr.visible = False

# move the layer
map.moveLayer(roads_major_buffers_sum_lyr, roads_major_points_along_lines_lyr, insert_position = "BEFORE")

True

#### <font color="darkgreen">Split Road Segments 1,000ft apart</font>

Split road segments at the points (1,000 feet apart)

In [114]:
# Create a path for the new split major roads feature class
roads_major_split_path = os.path.join(gdb_analysis_data_path, "roads_major_split")

# Split the major roads at the points along the lines
arcpy.management.SplitLineAtPoint(
    in_features = roads_major_path,
    point_features = roads_major_points_along_lines_path,
    out_feature_class = roads_major_split_path,
    search_radius = "1000 Feet"
)

<Result 'C:\\Users\\ktalexan\\OneDrive - County of Orange\\Documents\\OCSWITRS\\AGPSWITRS\\AGPSWITRS.gdb\\AnalysisData\\roads_major_split'>

Map layer operations

In [115]:
# Remove the newly created layer from map
map.removeLayer(map.listLayers("roads_major_split")[0])

# Change the feature class alias for the Split Major Roads
arcpy.AlterAliasName(roads_major_split_path, "Major Roads Split")

# Add the major road split layer to the hot spots map
roads_major_split_lyr = map_hotspots.addDataFromPath(roads_major_split_path)
roads_major_split_lyr.visible = False

# Apply the symbology for the major roads layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = roads_major_split_lyr,
    in_symbology_layer = os.path.join(layers_path, "Major Roads.lyrx"),
    symbology_fields = "VALUE_FIELD road_cat road_cat",
    update_symbology = "MAINTAIN"
)

<Result 'Major Roads Split'>

#### <font color="darkgreen">Buffers 500ft around road segments</font>

Create buffers (500 ft) around the road segments (1,000 feet)

In [116]:
# Create a path for the new split major roads feature class
roads_major_split_buffer_path = os.path.join(gdb_analysis_data_path, "roads_major_split_buffer")

# Buffer the split major roads by 500 feet
arcpy.analysis.Buffer(
    in_features = roads_major_split_path,
    out_feature_class = roads_major_split_buffer_path,
    buffer_distance_or_field = "500 Feet",
    line_side = "FULL",
    line_end_type = "FLAT",
    dissolve_option = "NONE",
    dissolve_field = None,
    method = "PLANAR"
)

<Result 'C:\\Users\\ktalexan\\OneDrive - County of Orange\\Documents\\OCSWITRS\\AGPSWITRS\\AGPSWITRS.gdb\\AnalysisData\\roads_major_split_buffer'>

Map layer operations

In [117]:
# Remove the newly created layer from map
map.removeLayer(map.listLayers("roads_major_split_buffer")[0])

# Change the feature class alias for the Split Major Roads Buffer
arcpy.AlterAliasName(roads_major_split_buffer_path, "Major Roads Split Buffer")

# Add the Major Roads Split Buffer layer to the hot spots map
roads_major_split_buffer_lyr = map_hotspots.addDataFromPath(roads_major_split_buffer_path)
roads_major_split_buffer_lyr.visible = False

# move the layer
map.moveLayer(roads_major_buffers_sum_lyr, roads_major_split_buffer_lyr, insert_position = "BEFORE")

True

#### <font color="darkgreen">Summarize road segments buffers</font>

Summarize the crash collision data for each of the road segments

In [118]:
# Create a path for the new summarized major road buffers feature class
roads_major_split_buffer_sum_path = os.path.join(gdb_analysis_data_path, "roads_major_split_buffer_sum")

# Summarize the data within the major road buffers from the crashes data
arcpy.analysis.SummarizeWithin(
    in_polygons = roads_major_split_buffer_path,
    in_sum_features = crashes_path,
    out_feature_class = roads_major_split_buffer_sum_path,
    keep_all_polygons = "KEEP_ALL",
    sum_fields=[
        ["crash_tag", "Sum"], ["party_count", "Sum"], ["victim_count", "Sum"], ["number_killed", "Sum"], 
        ["number_inj", "Sum"], ["count_severe_inj", "Sum"], ["count_visible_inj", "Sum"],
        ["count_complaint_pain", "Sum"], ["count_car_killed", "Sum"], ["count_car_inj", "Sum"],
        ["count_ped_killed", "Sum"], ["count_ped_inj", "Sum"], ["count_bic_killed", "Sum"],
        ["count_bic_inj", "Sum"], ["count_mc_killed", "Sum"], ["count_mc_inj", "Sum"],
        ["coll_severity_num", "Mean"], ["coll_severity_rank_num", "Mean"]
    ]
)

<Result 'C:\\Users\\ktalexan\\OneDrive - County of Orange\\Documents\\OCSWITRS\\AGPSWITRS\\AGPSWITRS.gdb\\AnalysisData\\roads_major_split_buffer_sum'>

Map layer operations

In [119]:
# Remove the newly created layer from map
map.removeLayer(map.listLayers("roads_major_split_buffer_sum")[0])

# Change the feature class alias for the Summarized Major Road Split Buffers
arcpy.AlterAliasName(roads_major_split_buffer_sum_path, "Major Roads Split Buffer Summary")

# Add the Major Roads Split Buffer layer to the hot spots map
roads_major_split_buffer_sum_lyr = map_hotspots.addDataFromPath(roads_major_split_buffer_sum_path)
roads_major_split_buffer_sum_lyr.visible = False

# move the layer
map.moveLayer(roads_major_split_buffer_lyr, roads_major_split_buffer_sum_lyr, insert_position = "BEFORE")

True

### <font color="blue">**2.8. Crashes within 500ft from Major Roads**</font>

Select all crashes that are within 500 ft of the major roads

In [127]:
# Create a path for the new feature class
crashes_500ft_from_major_roads_path = os.path.join(
    gdb_analysis_data_path, "Crashes_500ft_from_Major_Roads")

# Select the crashes within 500 feet of the major roads and store it in a temporary layer
temp_lyr = arcpy.management.SelectLayerByLocation(
    in_layer = crashes_path,
    overlap_type = "WITHIN_A_DISTANCE",
    select_features = roads_major_path,
    search_distance="500 Feet",
    selection_type="NEW_SELECTION",
    invert_spatial_relationship="NOT_INVERT"
)

Export the selected data to the geodatabase

In [128]:
# Export the selected crashes to a new feature class
arcpy.conversion.ExportFeatures(
    in_features = temp_lyr,
    out_features = crashes_500ft_from_major_roads_path,
    where_clause = "",
    use_field_alias_as_name = "NOT_USE_ALIAS",
)

<Result 'C:\\Users\\ktalexan\\OneDrive - County of Orange\\Documents\\OCSWITRS\\AGPSWITRS\\AGPSWITRS.gdb\\AnalysisData\\Crashes_500ft_from_Major_Roads'>

Map layer operations

In [129]:
# Delete the temporary layer
arcpy.management.Delete(temp_lyr)

# Remove the newly created layer from map
map.removeLayer(map.listLayers("Crashes_500ft_from_Major_Roads")[0])

# Change the feature class alias for the Crashes 500ft from Major Roads
arcpy.AlterAliasName(crashes_500ft_from_major_roads_path, "Crashes within 500ft from Major Roads")

# Add the new feature class to the map
crashes_500ft_from_major_roads_lyr = map_hotspots.addDataFromPath(crashes_500ft_from_major_roads_path)
crashes_500ft_from_major_roads_lyr.visible = False

# Apply the symbology for the Crashes 500ft from Major Roads data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = crashes_500ft_from_major_roads_lyr,
    in_symbology_layer = os.path.join(layers_path, "OCSWITRS Crashes.lyrx"),
    symbology_fields = "VALUE_FIELD coll_severity coll_severity",
    update_symbology = "MAINTAIN"
)

<Result 'Crashes within 500ft from Major Roads'>

Save the ArcGIS Pro project

In [130]:
# Save the project
aprx.save()


## <font color="orangered">**3. Hot Spot Analysis Operations**</font>

### <font color="blue">**3.1. Create Hot Spots (Crashes, Collision Severity)**</font>

Create a hot spot analysis for the crash collision data (collision severity)

In [131]:
# Create a path for the new crashes hot spots feature class
crashes_hot_spots_path = os.path.join(gdb_hotspot_data_path, "crashes_hot_spots")

# Create hot spots points
arcpy.stats.HotSpots(
    Input_Feature_Class = crashes_path,
    Input_Field = "coll_severity_num",
    Output_Feature_Class = crashes_hot_spots_path,
    Conceptualization_of_Spatial_Relationships = "FIXED_DISTANCE_BAND",
    Distance_Method = "EUCLIDEAN_DISTANCE",
    Standardization = "ROW",
    Distance_Band_or_Threshold_Distance = None,
    Self_Potential_Field = None,
    Weights_Matrix_File = None,
    Apply_False_Discovery_Rate__FDR__Correction = "NO_FDR",
    number_of_neighbors = None
)

<Result 'C:\\Users\\ktalexan\\OneDrive - County of Orange\\Documents\\OCSWITRS\\AGPSWITRS\\AGPSWITRS.gdb\\HotSpotData\\crashes_hot_spots'>

Map layer operations

In [132]:
# Remove the newly created layer from map
map.removeLayer(map.listLayers("crashes_hot_spots")[0])

# Change the feature class alias for the Crashes Hot Spots
arcpy.AlterAliasName(crashes_hot_spots_path, "Crashes Hot Spots")

# Add the new feature class to the map
crashes_hot_spots_lyr = map_hotspots.addDataFromPath(crashes_hot_spots_path)
crashes_hot_spots_lyr.visible = False

### <font color="blue">**3.2. Optimized Hot Spots (Crashes, Collision Severity, 1,000 m)**</font>

Optimized hot spot analysis for the crash collision data (collision severity)

In [133]:
# Create a path for the new optimized crashes hot spots feature class
crashes_optimized_hot_spots_path = os.path.join(gdb_hotspot_data_path, "crashes_optimized_hot_spots")

# Perform Optimized Hot Spot Analysis on the crashes data
arcpy.stats.OptimizedHotSpotAnalysis(
    Input_Features = crashes_path,
    Output_Features = crashes_optimized_hot_spots_path,
    Analysis_Field = "coll_severity_num",
    Incident_Data_Aggregation_Method = "COUNT_INCIDENTS_WITHIN_FISHNET_POLYGONS",
    Bounding_Polygons_Defining_Where_Incidents_Are_Possible = None,
    Polygons_For_Aggregating_Incidents_Into_Counts = None,
    Density_Surface = None,
    Cell_Size = None,
    Distance_Band = "1000 Meters"
)

<Result 'C:\\Users\\ktalexan\\OneDrive - County of Orange\\Documents\\OCSWITRS\\AGPSWITRS\\AGPSWITRS.gdb\\HotSpotData\\crashes_optimized_hot_spots'>

Map layer operations

In [134]:
# Remove the newly created layer from map
map.removeLayer(map.listLayers("crashes_optimized_hot_spots")[0])

# Change the feature class alias for the Optimized Crashes Hot Spots
arcpy.AlterAliasName(crashes_optimized_hot_spots_path, "Crashes Optimized Hot Spots")

# Add the new feature class to the map
crashes_optimized_hot_spots_lyr = map_hotspots.addDataFromPath(crashes_optimized_hot_spots_path)
crashes_optimized_hot_spots_lyr.visible = False


### <font color="blue">**3.3. Find Hot Spots (Crashes, 100m bins, 1km neighbors)**</font>

Find hot spots for the crash collision data using 100 m bins and 1 km neighborhood radius (328 ft/ 0.621 mi)

In [135]:
# Create a path for the new crashes find hot spots feature class
crashes_find_hot_spots_100m_1km_path = os.path.join(gdb_hotspot_data_path, "crashes_find_hot_spots_100m_1km")

# Find the hot spots within the crashes data
arcpy.gapro.FindHotSpots(
    point_layer = crashes_path,
    out_feature_class = crashes_find_hot_spots_100m_1km_path,
    bin_size = "100 Meters",
    neighborhood_size = "1 Kilometers",
    time_step_interval = None,
    time_step_alignment = "START_TIME",
    time_step_reference = None
)

<Result 'C:\\Users\\ktalexan\\OneDrive - County of Orange\\Documents\\OCSWITRS\\AGPSWITRS\\AGPSWITRS.gdb\\HotSpotData\\crashes_find_hot_spots_100m_1km'>

Map layer operations

In [136]:
# Remove the newly created layer from map
map.removeLayer(map.listLayers("crashes_find_hot_spots_100m_1km")[0])

# Change the feature class alias for the Crashes Find Hot Spots
arcpy.AlterAliasName(crashes_find_hot_spots_100m_1km_path, "Crashes Find Hot Spots 100m 1km")

# Add the new feature class to the map
crashes_find_hot_spots_100m_1km_lyr = map_hotspots.addDataFromPath(crashes_find_hot_spots_100m_1km_path)
crashes_find_hot_spots_100m_1km_lyr.visible = False


### <font color="blue">**3.4. Find Hot Spots (Crashes, 150m bins, 2km neighbors)**</font>

Find hot spots for the crash collision data using 150 m bins and 2 km neighborhood radius (492 ft/ 1.24 mi)

In [138]:
# Create a path for the new crashes find hot spots feature class
crashes_find_hot_spots_150m_2km_path = os.path.join(gdb_hotspot_data_path, "crashes_find_hot_spots_150m_2km")

# Find the hot spots within the crashes data
arcpy.gapro.FindHotSpots(
    point_layer = crashes_path,
    out_feature_class = crashes_find_hot_spots_150m_2km_path,
    bin_size = "150 Meters",
    neighborhood_size = "2 Kilometers",
    time_step_interval = None,
    time_step_alignment = "START_TIME",
    time_step_reference = None
)

<Result 'C:\\Users\\ktalexan\\OneDrive - County of Orange\\Documents\\OCSWITRS\\AGPSWITRS\\AGPSWITRS.gdb\\HotSpotData\\crashes_find_hot_spots_150m_2km'>

Map layer operations

In [139]:
# Remove the newly created layer from map
map.removeLayer(map.listLayers("crashes_find_hot_spots_150m_2km")[0])

# Change the feature class alias for the Crashes Find Hot Spots
arcpy.AlterAliasName(crashes_find_hot_spots_150m_2km_path, "Crashes Find Hot Spots 150m 2km")

# Add the new feature class to the map
crashes_find_hot_spots_150m_2km_lyr = map_hotspots.addDataFromPath(crashes_find_hot_spots_150m_2km_path)
crashes_find_hot_spots_150m_2km_lyr.visible = False

### <font color="blue">**3.5. Find Hot Spots (Crashes, 100m bins, 5km neighbors)**</font>

Find hot spots for the crash collision data using 100 m bins and 5 km neighborhood radius (328 ft/ 3.11 mi)

In [140]:
# Create a path for the new crashes find hot spots feature class
crashes_find_hot_spots_100m_5km_path = os.path.join(gdb_hotspot_data_path, "crashes_find_hot_spots_100m_5km")

# Find the hot spots within the crashes data
arcpy.gapro.FindHotSpots(
    point_layer = crashes_path,
    out_feature_class = crashes_find_hot_spots_100m_5km_path,
    bin_size = "100 Meters",
    neighborhood_size = "5 Kilometers",
    time_step_interval = None,
    time_step_alignment = "START_TIME",
    time_step_reference = None
)

<Result 'C:\\Users\\ktalexan\\OneDrive - County of Orange\\Documents\\OCSWITRS\\AGPSWITRS\\AGPSWITRS.gdb\\HotSpotData\\crashes_find_hot_spots_100m_5km'>

Map layer operations

In [141]:
# Remove the newly created layer from map
map.removeLayer(map.listLayers("crashes_find_hot_spots_100m_5km")[0])

# Change the feature class alias for the Crashes Find Hot Spots
arcpy.AlterAliasName(crashes_find_hot_spots_100m_5km_path, "Crashes Find Hot Spots 100m 5km")

# Add the new feature class to the map
crashes_find_hot_spots_100m_5km_lyr = map_hotspots.addDataFromPath(crashes_find_hot_spots_100m_5km_path)
crashes_find_hot_spots_100m_5km_lyr.visible = False

Save the ArcGIS Pro project

In [142]:
# Save the project
aprx.save()


### <font color="blue">**3.6. Hot Spots (Proximity to Major Roads, 500ft)**</font>

Hot spot points within 500 feet of major roads

In [143]:
# Create a path for the new hot spots within 500 mt from major roads feature class
crashes_within_500ft_from_major_roads_hot_spots_path = os.path.join(
    gdb_hotspot_data_path, "crashes_hot_spots_500ft_major_roads")

arcpy.stats.HotSpots(
    Input_Feature_Class = crashes_500ft_from_major_roads_path,
    Input_Field="coll_severity_num",
    Output_Feature_Class = crashes_within_500ft_from_major_roads_hot_spots_path,
    Conceptualization_of_Spatial_Relationships = "FIXED_DISTANCE_BAND",
    Distance_Method = "EUCLIDEAN_DISTANCE",
    Standardization = "ROW",
    Distance_Band_or_Threshold_Distance = None,
    Self_Potential_Field = None,
    Weights_Matrix_File = None,
    Apply_False_Discovery_Rate__FDR__Correction = "NO_FDR",
    number_of_neighbors = None
)

<Result 'C:\\Users\\ktalexan\\OneDrive - County of Orange\\Documents\\OCSWITRS\\AGPSWITRS\\AGPSWITRS.gdb\\HotSpotData\\crashes_hot_spots_500ft_major_roads'>

Map layer operations

In [144]:
# Remove the newly created layer from map
map.removeLayer(map.listLayers("crashes_hot_spots_500ft_major_roads")[0])

# Change the feature class alias for the Crashes Find Hot Spots
arcpy.AlterAliasName(
    crashes_within_500ft_from_major_roads_hot_spots_path, 
    "Crashes Hot Spots within 500ft from Major Roads")

# Add the new feature class to the map
crashes_within_500ft_from_major_roads_hot_spots_lyr = map_hotspots.addDataFromPath(crashes_within_500ft_from_major_roads_hot_spots_path)
crashes_within_500ft_from_major_roads_hot_spots_lyr.visible = False

### <font color="blue">**3.8. Find Hot Spots (Proximity to Major Roads, 500ft)**</font>

Find hot spots within 500 feet from major roads

In [145]:
# Create a path for the new hot spots within 500 ft from major roads feature class
crashes_within_500ft_from_major_roads_find_hot_spots_path = os.path.join(
    gdb_hotspot_data_path, "crashes_find_hot_spots_500ft_major_roads_500ft_1mi")

arcpy.gapro.FindHotSpots(
    point_layer = crashes_500ft_from_major_roads_path,
    out_feature_class = crashes_within_500ft_from_major_roads_find_hot_spots_path,
    bin_size="500 Feet",
    neighborhood_size = "1 Miles",
    time_step_interval = None,
    time_step_alignment = "START_TIME",
    time_step_reference=None
)

<Result 'C:\\Users\\ktalexan\\OneDrive - County of Orange\\Documents\\OCSWITRS\\AGPSWITRS\\AGPSWITRS.gdb\\HotSpotData\\crashes_find_hot_spots_500ft_major_roads_500ft_1mi'>

Map layer operations

In [146]:
# Remove the newly created layer from map
map.removeLayer(map.listLayers("crashes_find_hot_spots_500ft_major_roads_500ft_1mi")[0])

# Change the feature class alias for the Crashes Find Hot Spots
arcpy.AlterAliasName(
    crashes_within_500ft_from_major_roads_find_hot_spots_path, 
    "Crashes Hot Spots within 500ft from Major Roads 500ft 1mi")

# Add the new feature class to the map
crashes_within_500ft_from_major_roads_find_hot_spots_lyr = map_hotspots.addDataFromPath(crashes_within_500ft_from_major_roads_find_hot_spots_path)
crashes_within_500ft_from_major_roads_find_hot_spots_lyr.visible = False

## <font color="orangered">**4. Exploratory Regression Operations**</font>

### <font color="blue">**4.1. Crashes (Collision Severity) Exploratory Regression**</font>

In [ ]:
# Add collision severity binary indicator to crashes
arcpy.management.CalculateField(
    in_table = crashes_path,
    field = "severity_bin",
    expression = "sevbin(!coll_severity_bin!)",
    expression_type = "PYTHON3",
    code_block = """def sevbin(x):
    if x == "Severe or fatal":
        return 1
    elif x == "None, minor or pain":
        return 0""",
    field_type = "SHORT",
    enforce_domains = "NO_ENFORCE_DOMAINS"
)

Perform exploratory regression to predict the binary severity bin

In [ ]:
arcpy.stats.ExploratoryRegression(
    Input_Features=crashes_path,
    Dependent_Variable="severity_bin",
    Candidate_Explanatory_Variables="accident_year;coll_severity_num;coll_severity_rank_num;party_count;victim_count;number_killed;number_inj;count_severe_inj;count_visible_inj;count_complaint_pain;count_car_killed;count_car_inj;count_ped_killed;count_ped_inj;count_bic_killed;count_bic_inj;count_mc_killed;count_mc_inj",
    Weights_Matrix_File=None,
    Output_Report_File=None,
    Output_Results_Table=None,
    Maximum_Number_of_Explanatory_Variables=5,
    Minimum_Number_of_Explanatory_Variables=1,
    Minimum_Acceptable_Adj_R_Squared=0.5,
    Maximum_Coefficient_p_value_Cutoff=0.05,
    Maximum_VIF_Value_Cutoff=7.5,
    Minimum_Acceptable_Jarque_Bera_p_value=0.1,
    Minimum_Acceptable_Spatial_Autocorrelation_p_value=0.1
)

Save the ArcGIS Pro project

In [154]:
# Save the project
aprx.save()

## <font color="orangered">**5. Map Layout Operations**</font>

### <font color="blue">**4.1 Main Map Layout**</font>

Close all views and open the main layout and map

In [156]:
# Close all map views
aprx.closeViews()

# Open the main layout window
layout_main.openView()

# open the main map window
map_main.openView()

# set the main map as active map
map = aprx.activeMap